# Online marketplace analytics

Есть данные с бразильского маркетплейса. Они лежат в ClickHouse на удаленном сервере.

Таблицы:
* vk.orders
* vk.products
* vk.customers

Для получения описания столбцов можно использовать [DESCRIBE TABLE](https://clickhouse.tech/docs/ru/sql-reference/statements/misc/#misc-describe-table)
ClickHouse – колоночная база данных для хранения широких денормализованных таблиц. Поскольку у нас в день пишется в таблицу легко пишется >10кк строк, с ними необходимо работать максимально аккуратно. Поэтому введем следующие ограничения:
* Никаких SELECT *
* Максимально агрегировать данные в самом ClickHouse

Полезный референс по функциям [тут](https://clickhouse.tech/docs/ru/sql-reference/aggregate-functions/reference/).

Необходимо провести анализ покупок. Можно использовать любые данные и любые разрезы. Задание творческое, поэтому лучше всего ограничить себя по времени, чтобы не слишком зарываться.
Оформить в виде *.ipynb максимально просто, чтобы любой менеджер мог его понять.

In [ ]:
# Сюда вставить свои логин/пароль, полученные от HR
user = ''
password = ''

import requests
import pandas as pd
from io import StringIO
from requests.auth import HTTPBasicAuth

def get_df(query: str, user: str = user, password: str = password) -> pd.DataFrame:
    ''''''
    resp = requests.post(
        'https://clickhouse.kkmagician.com/', 
        data=(query + " FORMAT TSVWithNames").encode('utf-8'), 
        auth=HTTPBasicAuth(user, password)
    )

    if resp.status_code == 200:
        df = pd.read_csv(StringIO(resp.text), sep='\t')
        return df
    else:
        raise ValueError(resp.text)

In [ ]:
import plotly.express as px

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

## Описание таблиц

In [ ]:
get_df('describe vk.customers')

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,customer_id,UInt64,NaN,NaN,ID пользователя,NaN,NaN
1,customer_zip_code_prefix,Int32,NaN,NaN,Префикс индекса адреса пользователя. Можно исп...,NaN,NaN
2,customer_state,LowCardinality(String),NaN,NaN,"Штат, в котором проживает пользователь",NaN,NaN


In [ ]:
get_df('describe vk.orders')

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,order_id,UInt64,NaN,NaN,NaN,NaN,NaN
1,order_status,LowCardinality(String),NaN,NaN,"Последний статус заказа (т.к. это история, бол...",NaN,NaN
2,order_purchase_timestamp,DateTime,NaN,NaN,"Время, когда пользователь совершил заказ",NaN,NaN
3,order_approved_at,DateTime,NaN,NaN,"Время, когда магазин подтвердил заказ",NaN,NaN
4,order_delivered_carrier_date,DateTime,NaN,NaN,"Время, когда заказ был доставлен службой доставки",NaN,NaN
5,order_delivered_customer_date,DateTime,NaN,NaN,"Время, когда доставка была подтверждена пользо...",NaN,NaN
6,order_estimated_delivery_date,DateTime,NaN,NaN,Оценка даты доставки заказа на момент его сове...,NaN,NaN
7,customer_id,UInt64,NaN,NaN,NaN,NaN,NaN
8,order_item_id,UInt16,NaN,NaN,Порядковый номер товара в заказе,NaN,NaN
9,product_id,UInt64,NaN,NaN,NaN,NaN,NaN


In [ ]:
get_df('describe vk.products')

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,product_id,UInt64,NaN,NaN,NaN,NaN,NaN
1,product_name_length,UInt16,NaN,NaN,Длина названия товара в символах,NaN,NaN
2,product_description_length,UInt32,NaN,NaN,Длина описания товара в символах,NaN,NaN
3,product_photos_qty,UInt16,NaN,NaN,Количество фото у товара,NaN,NaN
4,product_weight_g,UInt32,NaN,NaN,NaN,NaN,NaN
5,product_length_cm,UInt32,NaN,NaN,NaN,NaN,NaN
6,product_height_cm,UInt32,NaN,NaN,NaN,NaN,NaN
7,product_width_cm,UInt32,NaN,NaN,NaN,NaN,NaN
8,product_category_name,LowCardinality(String),NaN,NaN,Категория товара,NaN,NaN


## Аудитория маркетплейса

In [ ]:
#Запрос: Среднее число клиентов по штатам
cust_by_state = get_df('select customer_state,\
                               uniqExact(customer_id) as num_cust\
                        from vk.customers\
                        group by customer_state\
                        order by num_cust desc')

In [ ]:
#График: Среднее число клиентов по штатам
fig = px.bar(cust_by_state, x='customer_state', y='num_cust', labels={
                     "customer_state": "Штат",
                     "num_cust": "Число покупателей"
                 }, title = 'Среднее число покупателей по штатам')
fig.show()

In [ ]:
#Число активных клиентов в месяц (данные)
MAU_all = get_df('select toStartOfMonth(r.order_purchase_timestamp) as so_month,\
                         uniqExact(l.customer_id) as num_cust\
                  from vk.customers as l\
                  join vk.orders as r\
                  on l.customer_id = r.customer_id\
                  group by so_month\
                  order by so_month asc')

In [ ]:
#Число активных клиентов в месяц (график)
fig = px.line(MAU_all[MAU_all.so_month < '2018-09-01'], x="so_month", y="num_cust",\
              labels={
                     "so_month": "",
                     "num_cust": "Число покупателей"
                 }, title='Число активных покупателей по месяцам')
fig.show()

In [ ]:
#Число активных покупателей по штатам (данные)
MAU_by_states = get_df('select toStartOfMonth(r.order_purchase_timestamp) as so_month,\
                                l.customer_state as state,\
                                 uniqExact(l.customer_id) as num_cust\
                          from vk.customers as l\
                          join vk.orders as r\
                          on l.customer_id = r.customer_id\
                          group by state, so_month\
                          order by so_month asc, num_cust desc')

In [ ]:
#Возьмем данные только по штатам с наибольшим числом покупателей
MAU_top_states = MAU_by_states.query('state in ["SP", "RJ", "MG"]')

In [ ]:
#И посмотрим на динамику числа клиентов в этих штатах
fig = px.line(MAU_top_states[MAU_top_states.so_month < '2018-09-01'], x="so_month", y="num_cust", color = 'state',\
              labels={
                     "so_month": "",
                     "num_cust": "Число покупателей",
                     "state":"Штат"
                 },
              title='Динамика числа покупателей в Топ-3 штатах по числу покупателей')
fig.show()

In [ ]:
#Также проверим динамику дневной аудитории маркетплейса
DAU_all = get_df('select toStartOfDay(r.order_purchase_timestamp) as so_day,\
                         uniqExact(l.customer_id) as num_cust\
                  from vk.customers as l\
                  join vk.orders as r\
                  on l.customer_id = r.customer_id\
                  group by so_day\
                  order by so_day asc')

In [ ]:
fig = px.line(DAU_all[DAU_all.so_day < '2018-09-01'], x="so_day", y="num_cust",\
              labels={
                     "so_day": "",
                     "num_cust": "Число покупателей"
                 },title='Число покупателей в день')
fig.show()

В динамике по дням уже можно видеть пик более 1000 покупателей в день 1 ноября при всех значениях, не превышающих 400 чел. Проверим, есть ли этот пик в Топ-3 штатах.

In [ ]:
#Аналогичным образом получим данные в динамике в разбивке по штатам
DAU_by_states = get_df('select toStartOfDay(r.order_purchase_timestamp) as so_day,\
                                l.customer_state as state,\
                                 uniqExact(l.customer_id) as num_cust\
                          from vk.customers as l\
                          join vk.orders as r\
                          on l.customer_id = r.customer_id\
                          group by state, so_day\
                          order by so_day asc, num_cust desc')

In [ ]:
#Выберем Топ-3 штата
DAU_top_states = DAU_by_states.query('state in ["SP", "RJ", "MG"]')

In [ ]:
#Построим график
fig = px.line(DAU_top_states[DAU_top_states.so_day < '2018-09-01'], x="so_day", y="num_cust", color = 'state',\
              labels={
                     "so_day": "",
                     "num_cust": "Число покупателей",
                     "state":"Штат"
                 },
              title='Число покупателей в день в Топ-3 штатах по числу клиентов')
fig.show()

По графику выше можно видеть, что рост 24 ноября 2017 произошел во всех Топ-3 штатах, но сильнее всего в самом многочисленном.

In [ ]:
# Средняя дневная аудитория (запрос)
DAU_avg = get_df('select toStartOfMonth(so_day) as so_month,\
                        avg(num_cust) as avg_num_cust_daily\
                from \
               (select toStartOfDay(r.order_purchase_timestamp) as so_day,\
                                 uniqExact(l.customer_id) as num_cust\
                          from vk.customers as l\
                          join vk.orders as r\
                          on l.customer_id = r.customer_id\
                          group by so_day\
                          order by so_day asc)\
               group by so_month\
               order by so_month asc')

In [ ]:
fig = px.line(DAU_avg[DAU_avg.so_month < '2018-09-01'], x="so_month", y="avg_num_cust_daily",\
              labels={
                     "so_month": "",
                     "avg_num_cust_daily": "Среднемесячная дневная аудитория"
                 }, title='Среднее ежедневное число активных покупателей по месяцам')
fig.show()

In [ ]:
# Объединим данные по ср. DAU и MAU
dau_mau_data = pd.merge(DAU_avg, MAU_all, on = 'so_month')

In [ ]:
# Посчитаем stickness ratio
dau_mau_data['stick_ratio'] = dau_mau_data['avg_num_cust_daily']/dau_mau_data['num_cust'] * 100

In [ ]:
#Построим график stickness ratio
fig = px.line(dau_mau_data[(dau_mau_data.so_month < '2018-09-01') & (dau_mau_data.so_month >= '2017-01-01')],\
              x="so_month", y="stick_ratio",\
              labels={
                     "so_month": "",
                     "stick_ratio": "Stickness ratio"
                 },
              title='Процент уникальных ежедневных покупателей, остающихся в течение месяца')
fig.show()

Также посмотрим, как изменялся размер аудитории в разбивке по продавцам, т.е. какое число покупателей приходилось на продавца в месяц.

In [ ]:
#Запрос: Динамика соотношения числа клиентов и продавцов
cl_sel_ratio_mo = get_df('select toStartOfMonth(order_purchase_timestamp) as so_month,\
                                 uniqExact(customer_id)/uniqExact(seller_id) as cl_sel_ratio\
                        from vk.orders\
                        group by so_month\
                        order by so_month asc')

In [ ]:
#График: Динамика соотношения числа клиентов и продавцов
fig = px.line(cl_sel_ratio_mo[cl_sel_ratio_mo.so_month < '2018-09-01'], x='so_month', y='cl_sel_ratio',\
              labels={
                     "so_month": "",
                     "cl_sel_ratio": "Число покупателей на одного продавца"},\
             title = 'Динамика соотношения числа клиентов и продавцов')
fig.show()

In [ ]:
cl_sel_ratio_mo.cl_sel_ratio.mean()

5.042399952449101

In [ ]:
#Cоотношение числа клиентов и продавцов по штатам
cl_sel_ratio_st = get_df('select r.customer_state as state,\
                                 uniqExact(l.customer_id)/uniqExact(l.seller_id) as cl_sel_ratio\
                        from vk.orders as l\
                        join vk.customers as r\
                        on l.customer_id = r.customer_id\
                        group by state\
                        order by cl_sel_ratio desc')

In [ ]:
#График: Cоотношение числа клиентов и продавцов по штатам
fig = px.bar(cl_sel_ratio_st, x='state', y='cl_sel_ratio',\
             labels={
                     "state": "Штат",
                     "cl_sel_ratio": "Число покупателей на одного продавца"},\
             title = 'Cоотношение числа клиентов и продавцов по штатам')
fig.show()

По графику выше видно, что продавцы-лидеры по охвату аудитории находятся в штатах с наибольшим числом клиентов и заказов.

## Число заказов

In [ ]:
#Число заказов по месяцам
Num_ord_month = get_df('select toStartOfMonth(order_purchase_timestamp) as so_month,\
                               uniqExact(order_id) as num_ord\
                        from vk.orders\
                        group by so_month\
                        order by so_month asc')

In [ ]:
#График: Число заказов по месяцам
fig = px.line(Num_ord_month[Num_ord_month.so_month < '2018-09-01'], x='so_month', y='num_ord',\
              labels={
                     "so_month": "",
                     "num_ord": "Число заказов"},\
             title = 'Число заказов по месяцам')
fig.show()

In [ ]:
#Число заказов по штатам
Num_ord_state = get_df('select uniqExact(l.order_id) as num_ord,\
                               r.customer_state as state\
                        from vk.orders as l\
                        join vk.customers as r\
                        on l.customer_id = r.customer_id\
                        group by state\
                        order by num_ord desc')

In [ ]:
#График: Число заказов по штатам
fig = px.bar(Num_ord_state, x='state', y='num_ord',
             labels={
                     "state": "Штат",
                     "num_ord": "Число заказов"},\
             title = 'Число заказов по штатам')
fig.show()

Как по числу заказов, так и по числу покупателей лидируют одни и те же штаты.

## Средний размер заказа и выручка

In [ ]:
#Средний размер заказа по штатам
AoV_by_state = \
get_df('select state,\
               avg(sum_order) as avg_order\
        from \
        (select order_id,\
               state,\
               sum(price*quant) as sum_order\
        from \
       (select l.order_id as order_id,\
                   l.product_id as product_id,\
                   r.customer_state as state,\
                   l.price as price,\
                   count(l.product_id) as quant\
            from vk.orders as l\
            join vk.customers as r\
            on l.customer_id = r.customer_id\
            group by l.order_id,\
                     l.product_id,\
                     r.customer_state,\
                     l.price\
            order by l.order_id, quant desc)\
         group by state, order_id)\
         group by state\
         order by avg_order desc')

In [ ]:
#Средний размер заказа по штатам
fig = px.bar(AoV_by_state, x='state', y='avg_order',\
             labels={
                     "state": "Штат",
                     "avg_order": "Средний размер заказа"},\
             title = 'Средний размер заказа по штатам')
fig.show()

В среднем, штаты не сильно отличаются по размеру заказа, но лидеры по среднему размеру заказа отличаются от лидеров по количеству покупателей и заказов.

In [ ]:
#запрос SQL: Средний размер заказа по месяцам
AoV_by_month = \
get_df('select so_month,\
               avg(sum_order) as avg_order\
        from \
            (select order_id,\
                   so_month,\
                   sum(price*quant) as sum_order\
            from \
               (select order_id,\
                       product_id,\
                       price,\
                       count(product_id) as quant,\
                       toStartOfMonth(order_purchase_timestamp) as so_month\
                from vk.orders\
                group by order_id,\
                         product_id,\
                         price,\
                         so_month\
                order by order_id, quant desc)\
            group by so_month, order_id)\
         group by so_month\
         order by so_month asc')

In [ ]:
#График: Средний размер заказа по месяцам
fig = px.line(AoV_by_month, x="so_month", y="avg_order",\
              labels={
                     "so_month": "",
                     "avg_order": "Средний размер заказа"},\
              title='Средний размер заказа по месяцам')
fig.show()

In [ ]:
#Размер выручки по месяцам
Month_rev = \
get_df('select so_month,\
               sum(sum_order) as revenue\
        from \
           (select so_month,\
                       sum(price*quant) as sum_order\
            from \
                   (select order_id,\
                           product_id,\
                           price,\
                           count(product_id) as quant,\
                           toStartOfMonth(order_purchase_timestamp) as so_month\
                    from vk.orders\
                    group by order_id,\
                             product_id,\
                             price,\
                             so_month\
                    order by order_id, quant desc)\
             group by so_month, order_id\
             order by so_month asc)\
         group by so_month\
         order by so_month')

In [ ]:
fig = px.line(Month_rev[Month_rev.so_month < '2018-09-01'], x='so_month', y='revenue',\
              labels={
                     "so_month": "",
                     "revenue": "Размер выручки"},\
             title = 'Размер выручки по месяцам')
fig.show()

## Ассортимент

Посчитаем среднее число товаров по категориям

In [ ]:
#Динамика соотношения числа товаров и категорий
divers_ratio_month = get_df('select toStartOfMonth(l.order_purchase_timestamp) as so_month,\
                                   uniqExact(l.product_id)/uniqExact(r.product_category_name) as divers_ratio\
                            from vk.orders as l\
                            join vk.products as r\
                            on l.product_id = r.product_id\
                            group by so_month\
                            order by so_month asc')

In [ ]:
#График: Динамика соотношения числа товаров и категорий
fig = px.line(divers_ratio_month[divers_ratio_month.so_month < '2018-09-01'], x='so_month', y='divers_ratio',\
              labels={
                     "so_month": "",
                     "divers_ratio": "Число товаров в категории"},\
             title = 'Динамика соотношения числа товаров и категорий')
fig.show()

## Выводы
* **Аудитория**: посмотрели общее число покупателей, MAU, DAU, в динамике и по штатам, также среднемесячное DAU, stickness ratio и соотношение покупателей к продавцам в динамике. 
    1. В разбивке по штатам есть явные лидеры по общему числу покупателей: SP с >40к покупателей, и два других штата с 11-12к покупателей за весь период. 
    2. Число уникальных покупателей в мес. (MAU) выросло ~ в 20 раз с начала периода, в самом многочисленном штате  - в ~30 раз. 
    3. Однако среднее ежедневное число покупателей (ср. DAU) росло более медленно, stickness ratio (ср. DAU/MAU * 100) составляло примерно 3,2-3,7% (после исключения аномальных месяцев с 1 заказом.мес.). Т.е. в среднем активные пользователи не очень часто использовали продукт, также этот показатель практически не растет, что указывает на низкий потенциал роста прибили в будущем.
    4. В среднем на одного продавца приходилось 5 покупателей, и этот показатель в последнее время снижается, что указывает на снижение аудитории каждого продавца и потенциальное снижение прибыли.
* **Заказы**: 
    1. В среднем, число заказов растет, по штатам лидеры по числу заказов те же самые. 
    2. Однако по среднему размеру заказа (AoV) лидируют другие штаты и в целом, размер заказа отличается не сильно между штатами. 
    3. Исходя из анализа аудитории, нужно либо наращивать число заказов в остальных штатах (помимо топ-3), либо увеличивать средний размер заказа в топ-3 по числу заказов. Также нужно улучшать опыт пользователя, чтобы было большее число повторных покупок. 
* **Выручка**:
    Выручка доросла до почти 1млн и начала снижаться в последние 3 мес. Возможно, это является снижением числа повторных активных пользователей, т.е. нужно улучшать опыт пользователя, чтобы не было дальнейшего снижения.
* **Ассортимент**:
    Число товаров в одной категории доросло почти до 70 шт. в категории, что в целом говорит о росте выбора для покупателей, в целом ничего здесь вероятно менять не нужно.